# Nutrisense

In [ ]:
! pip install scikit-learn

In [11]:
import pandas as pd

# Load dataset
file_path = 'recipe_final1.csv'
recipe_df = pd.read_csv(file_path)

recipe_df.head()

,Unnamed: 0.1,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,0,0,222388,Homemade Bacon,5.00,https://images.media-allrecipes.com/userphotos...,3,15,36,1,42,21,81,2,"['pork belly', 'smoked paprika', 'kosher salt'..."
1,2,2,218939,Foolproof Rosemary Chicken Wings,4.57,https://images.media-allrecipes.com/userphotos...,12,17,36,2,48,24,31,4,"['chicken wings', 'sprigs rosemary', 'head gar..."
2,7,7,244856,Turkey Black Bean Burgers,4.86,https://images.media-allrecipes.com/userphotos...,7,11,14,2,51,28,11,11,"['extra lean ground turkey', 'cooked black bea..."
3,10,10,23658,Pam's Bierocks,4.63,https://images.media-allrecipes.com/userphotos...,130,15,15,13,19,11,17,8,"['warm water', 'active dry yeast', 'white suga..."
4,11,11,244786,Zucchini Noodles and Summer Vegetables with Sw...,4.50,https://images.media-allrecipes.com/userphotos...,2,10,21,3,23,13,56,8,"['zucchini cut into noodle-shape strands', 'ex..."


In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [53]:
# Preprocess bahan
vectorizer = TfidfVectorizer()
X_ingredients = vectorizer.fit_transform(recipe_df['ingredients_list'])

In [34]:
# Normalisasi data numerik
scaler = StandardScaler()
X_numerical = scaler.fit_transform(recipe_df[['protein', 'fat', 'fiber']])

In [54]:
# gabungkan fitur
# X_combined = np.hstack([X_numerical, X_ingredients.toarray()])

# Train KNN Model dgn n = 3
nn = NearestNeighbors(n_neighbors=5, metric='euclidean')
nn.fit(X_ingredients.toarray())

NearestNeighbors(metric='euclidean')

In [41]:
import pickle
pickle.dump(nn, open('reduced_recommendation_model2.pkl' , 'wb'))
pickle.dump(scaler, open('scaler2.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer2.pkl', 'wb'))

In [3]:
import pickle
knn = pickle.load(open('reduced_recommendation_model.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))

c:\Users\ergik\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator NearestNeighbors from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ergik\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ergik\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estim

In [16]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Function to Recommend Recipes
def recommend_recipes(umur, gender, list_bahan):
    data_nutrisi = [[13.3, 18.3, 7.8], #kategori 0
                      [16.7, 21.7, 9.3], #kategori 1
                      [18.3, 21.7, 9  ]] #kategori 2
    
        #mengategorikan berdasarkan umur dan gender
        #dikhusus untuk anak sd range 7-12
    if umur < 7:
        print("maaf umur tidak termasuk anak sd")
    elif umur < 10:
        kategori = 0
    elif umur < 13:
        if gender == 1: #1 untuk laki laki dan 0 untuk perempuan
            kategori = 1
        else:
            kategori = 2
    else:
        print("maaf umur tidak termasuk anak sd")
    
    menu_bergizi1 = recipe_df[recipe_df.protein > data_nutrisi[kategori][0]]
    menu_bergizi2 = menu_bergizi1[menu_bergizi1.fat > data_nutrisi[kategori][1]]
    menu_bergizi3 = menu_bergizi2[menu_bergizi2.fiber > data_nutrisi[kategori][2]]

    vectorizer = TfidfVectorizer()
    X_ingredients = vectorizer.fit_transform(menu_bergizi3['ingredients_list'])
    nn = NearestNeighbors(n_neighbors=5, metric='euclidean')
    nn.fit(X_ingredients.toarray())
    # merekomendasikan resep
    input_ingredients_transformed = vectorizer.transform([list_bahan])
    distances, indices = nn.kneighbors(input_ingredients_transformed.toarray())
    recommendations = menu_bergizi3.iloc[indices[0]]

    return recommendations[['recipe_name', 'ingredients_list', 'image_url']]

# Example Input
recommendations = recommend_recipes(10, 0,'beef')
recommendations

,recipe_name,ingredients_list,image_url
1007,Hamburger Hash,"['ground beef', 'large onion', 'potatoes', 'be...",https://images.media-allrecipes.com/userphotos...
3989,Easy Beef Strogonoff,"['ground beef', 'garlic powder', 'salt', 'grou...",https://images.media-allrecipes.com/userphotos...
2431,Beef Heart Braised in Wine,"['beef heart', 'all-purpose flour', 'salt and ...",https://images.media-allrecipes.com/userphotos...
722,Beef Pot Pie II,"['peeled and cubed red potatoes', 'carrots', '...",https://images.media-allrecipes.com/userphotos...
1930,Beef and Rice Medley,"['ground beef', 'diced onion', 'water', 'uncoo...",https://images.media-allrecipes.com/userphotos...


In [9]:
import pandas as pd

# Load dataset
file_path = 'data_nutrisi.csv'
nutrisi = pd.read_csv(file_path)

nutrisi.head()

,kategori,Kelompok Umur,Protein,Lemak,Karbohidrat,Serat
0,1,1 – 3 tahun,20,45,215,19
1,2,4 – 6 tahun,25,50,220,20
2,3,7 – 9 tahun,40,55,250,23
3,4,10 – 12 tahun,50,65,300,28
4,5,13 – 15 tahun,70,80,350,34


In [1]:
import pandas as pd

# Load dataset
file_path = 'reduced_dataset.csv'
recipe_df = pd.read_csv(file_path)

recipe_df.head()

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,19591,245720,Churro Log Cabin,5.00,http://images.media-allrecipes.com/userphotos/...,1,77,70,91,10,2,15,9,"['canned white frosting', ""pre-made churros (s..."
1,11552,247451,Pulled BBQ Chicken Sliders,4.17,https://images.media-allrecipes.com/userphotos...,5,20,20,110,32,12,30,4,"['skinless boneless chicken breast halves', 'o..."
2,4780,232860,Easy Tasty Ham Steaks with Maple Glaze For 2,4.62,https://images.media-allrecipes.com/userphotos...,19,52,18,71,36,15,65,10,"['ham steak', 'brown sugar', 'maple syrup', 'D..."
3,24585,245028,Poached Pears Belle Helene,5.00,http://images.media-allrecipes.com/userphotos/...,7,53,21,78,8,5,10,35,"['water', 'lemon zested and juiced', 'Bosc pea..."
4,20998,232909,American Flag Cake,4.22,http://images.media-allrecipes.com/userphotos/...,38,55,39,69,21,3,31,2,"['white cake mix', 'egg whites', 'vegetable oi..."


In [10]:
print(max(recipe_df["protein"]),
      max(recipe_df["fat"]),
      max(recipe_df["carbohydrates"]),
      max(recipe_df["fiber"]))

171 150 161 86


In [15]:
print(recipe_df["protein"].mean() ,
      recipe_df["fat"].mean(),
      recipe_df["carbohydrates"].mean(),
      recipe_df["fiber"].mean())

30.8544 33.4258 25.4254 17.046


In [12]:
print(min(recipe_df["protein"]),
      min(recipe_df["fat"]),
      min(recipe_df["carbohydrates"]),
      min(recipe_df["fiber"]))

0 0 18 0


In [1]:
import pandas as pd

# Load dataset
file_path = 'recipe_final.csv'
recipe = pd.read_csv(file_path)

recipe.head()

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,0,222388,Homemade Bacon,5.00,https://images.media-allrecipes.com/userphotos...,3,15,36,1,42,21,81,2,"['pork belly', 'smoked paprika', 'kosher salt'..."
1,1,240488,"Pork Loin, Apples, and Sauerkraut",4.76,https://images.media-allrecipes.com/userphotos...,29,19,18,10,73,33,104,41,"['sauerkraut drained', 'Granny Smith apples sl..."
2,2,218939,Foolproof Rosemary Chicken Wings,4.57,https://images.media-allrecipes.com/userphotos...,12,17,36,2,48,24,31,4,"['chicken wings', 'sprigs rosemary', 'head gar..."
3,3,87211,Chicken Pesto Paninis,4.62,https://images.media-allrecipes.com/userphotos...,163,32,45,20,65,20,43,18,"['focaccia bread quartered', 'prepared basil p..."
4,4,245714,Potato Bacon Pizza,4.50,https://images.media-allrecipes.com/userphotos...,2,8,12,5,14,7,8,3,"['red potatoes', 'strips bacon', 'Sauce:', 'he..."


In [28]:
print(max(recipe["protein"]),
      max(recipe["fat"]),
      max(recipe["carbohydrates"]),
      max(recipe["fiber"]),
      max(recipe["calories"]))

920 591 280 178 235


In [13]:
print(min(recipe["protein"]),
      min(recipe["fat"]),
      min(recipe["carbohydrates"]),
      min(recipe["fiber"]))

0 0 0 0


In [16]:
print(recipe["protein"].mean() ,
      recipe["fat"].mean(),
      recipe["carbohydrates"].mean(),
      recipe["fiber"].mean())

27.65193392838822 25.601149071509184 10.152190417564379 11.03892479737355


In [43]:
s1 = recipe[recipe.protein>25]
s2 = s1[s1.fat>28]
s3 = s2[s2.fiber>6]
s3.count() #highest

Unnamed: 0          7523
recipe_id           7523
recipe_name         7523
aver_rate           7523
image_url           7523
review_nums         7523
calories            7523
fat                 7523
carbohydrates       7523
protein             7523
cholesterol         7523
sodium              7523
fiber               7523
ingredients_list    7523
dtype: int64

In [2]:
s1 = recipe[recipe.protein>7]
s2 = s1[s1.fat>13]
new_reduced = s2[s2.fiber>6]
new_reduced.count() #lowest

Unnamed: 0          18038
recipe_id           18038
recipe_name         18038
aver_rate           18038
image_url           18038
review_nums         18038
calories            18038
fat                 18038
carbohydrates       18038
protein             18038
cholesterol         18038
sodium              18038
fiber               18038
ingredients_list    18038
dtype: int64

In [31]:
z1 = new_reduced[recipe.protein<25]
z2 = s1[s1.fat<28]
recipe_df = s2[s2.fiber<12]
recipe_df.count() #highest

C:\Users\ergik\AppData\Local\Temp\ipykernel_2760\2128814795.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  z1 = new_reduced[recipe.protein<25]


Unnamed: 0          15947
recipe_id           15947
recipe_name         15947
aver_rate           15947
image_url           15947
review_nums         15947
calories            15947
fat                 15947
carbohydrates       15947
protein             15947
cholesterol         15947
sodium              15947
fiber               15947
ingredients_list    15947
dtype: int64

In [10]:
recipe_df

,Unnamed: 0.1,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,0,0,222388,Homemade Bacon,5.00,https://images.media-allrecipes.com/userphotos...,3,15,36,1,42,21,81,2,[]
1,2,2,218939,Foolproof Rosemary Chicken Wings,4.57,https://images.media-allrecipes.com/userphotos...,12,17,36,2,48,24,31,4,[]
2,7,7,244856,Turkey Black Bean Burgers,4.86,https://images.media-allrecipes.com/userphotos...,7,11,14,2,51,28,11,11,[]
3,10,10,23658,Pam's Bierocks,4.63,https://images.media-allrecipes.com/userphotos...,130,15,15,13,19,11,17,8,[]
4,11,11,244786,Zucchini Noodles and Summer Vegetables with Sw...,4.50,https://images.media-allrecipes.com/userphotos...,2,10,21,3,23,13,56,8,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15942,48691,48691,20280,Coconut Puffs,4.14,https://images.media-allrecipes.com/userphotos...,6,28,62,14,11,0,8,9,[]
15943,48711,48711,216740,Cream Cheese Alfredo Sauce,4.63,https://images.media-allrecipes.com/userphotos...,67,23,64,2,34,41,28,2,[]
15944,48713,48713,32615,Cappuccino on Ice,4.73,https://images.media-allrecipes.com/userphotos...,57,11,14,10,11,11,3,0,[]
15945,48714,48714,246927,Italian Tofu Meatballs,4.50,https://images.media-allrecipes.com/userphotos...,1,17,36,6,32,18,37,9,[]


In [4]:
recipe_df.to_csv('recipe_final1.csv') 

In [13]:
recipe_df['ingredients_list']

0        ['pork belly', 'smoked paprika', 'kosher salt'...
1        ['chicken wings', 'sprigs rosemary', 'head gar...
2        ['extra lean ground turkey', 'cooked black bea...
3        ['warm water', 'active dry yeast', 'white suga...
4        ['zucchini cut into noodle-shape strands', 'ex...
                               ...                        
15942    ['all-purpose flour', 'self-rising flour', 'gr...
15943    ['butter', 'portobello mushroom caps', 'cream ...
15944    ['strong brewed coffee', 'sweetened condensed ...
15945    ['extra-firm tofu', 'onion', 'egg', 'bread cru...
15946    ['butter', 'habanero peppers', 'fresh peaches'...
Name: ingredients_list, Length: 15947, dtype: object

In [27]:
bahan = list(recipe_df['ingredients_list'][0])

In [28]:
bahan

['[',
 "'",
 'p',
 'o',
 'r',
 'k',
 ' ',
 'b',
 'e',
 'l',
 'l',
 'y',
 "'",
 ',',
 ' ',
 "'",
 's',
 'm',
 'o',
 'k',
 'e',
 'd',
 ' ',
 'p',
 'a',
 'p',
 'r',
 'i',
 'k',
 'a',
 "'",
 ',',
 ' ',
 "'",
 'k',
 'o',
 's',
 'h',
 'e',
 'r',
 ' ',
 's',
 'a',
 'l',
 't',
 "'",
 ',',
 ' ',
 "'",
 'g',
 'r',
 'o',
 'u',
 'n',
 'd',
 ' ',
 'b',
 'l',
 'a',
 'c',
 'k',
 ' ',
 'p',
 'e',
 'p',
 'p',
 'e',
 'r',
 "'",
 ']']

In [29]:
all_bahan = []
for i in recipe_df['ingredients_list']:
    print(type(i))
    # for item in i:
    #     all_bahan.append(recipe_df[item]['ingredients_list'])
    #     print()
all_bahan

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

[]